# **Reportes automáticos para saldos de clientes**

 Este programa sirve para **generar reportes** de forma **automática** a partir de un **archivo excel**.

 Cuando se ejecuta, toma todos los clientes presentes en el archivo y devuelve un PDF con un resumen de su saldo, acompañado por una gráfica y una tabla. Por supuesto, pueden añadirse cálculos y funciones a este código para que el output tenga la complejidad necesaria para su empresa.

 El excel que aparece como ejemplo ha sido creado por IA, tratándose de distribuidoras y números ficticios. Para realizar el código, trabajé desde **Google Colab**, por lo tanto, las rutas corresponden a mi unidad y carpetas. En caso de emplear otra interfaz, las líneas de **montar google drive** son **innecesarias** y pueden sustituirse por:

 ▶ *df = pd.read_excel("ruta_del_archivo.xlsx")*

In [ ]:
#En caso de no tener estos paquetes, debe instalarlos en su entorno de trabajo.
#!pip install pandas plotly jinja2 xhtml2pdf
#!pip install -U kaleido

In [ ]:
import os
import pandas as pd
import plotly.express as px
from jinja2 import Template, Environment
from xhtml2pdf import pisa
from datetime import datetime

# Crear un filtro personalizado para formatear números con comas
def format_number(value):
    return f"{value:,.2f}"

# Cargar el template y registrar el filtro
env = Environment()
env.filters['format_number'] = format_number

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Leer archivo
ruta = "/content/drive/MyDrive/Colab Notebooks/ventas_ceramicas.xlsx"
df = pd.read_excel(ruta)

# Agregar columna de saldo
df["Saldo"] = df["Importe"] - df["Pagado"]
clientes = df["Cliente"].unique()

# Crear la carpeta en la ubicación correcta
reportes_path = "/content/drive/MyDrive/Colab Notebooks/reportes_clientes"
os.makedirs(reportes_path, exist_ok=True)

# Confirmar si la carpeta fue creada correctamente
print(f"Carpeta creada en: {reportes_path}")

# Generar reportes para cada cliente
for cliente in clientes:
    print(f"Generando reporte para el cliente: {cliente}")
    datos = df[df["Cliente"] == cliente]
    total_importe = datos["Importe"].sum()
    total_pagado = datos["Pagado"].sum()
    total_saldo = datos["Saldo"].sum()

    # Crear gráfico con Plotly
    fig = px.bar(datos, x="Producto", y=["Importe", "Pagado"], barmode="group",
                 title="Importe vs Pagado", color_discrete_sequence=["#4472C4", "#A9D18E"])

    # Guardar el gráfico como imagen
    grafico_path = f"{reportes_path}/{cliente}_grafico.png"
    fig.write_image(grafico_path, width=800, height=400)

    # Confirmar que el gráfico fue guardado
    print(f"Gráfico guardado en: {grafico_path}")

    # Crear tabla en HTML
    tabla = datos[["Producto", "Fecha Venta", "Importe", "Pagado", "Saldo"]].copy()
    tabla["Fecha Venta"] = tabla["Fecha Venta"].dt.strftime("%Y-%m-%d")
    tabla_html = tabla.to_html(index=False, classes='styled-table', border=0)

    # Template HTML con el filtro de formateo de número
    template_html = env.from_string("""
    <html>
    <head><meta charset="utf-8">
    <style>
    body { font-family: Helvetica, sans-serif; padding: 20px; }
    h1 { color: #4472C4; }
    .styled-table {
        width: 100%;
        border-collapse: collapse;
        margin-top: 20px;
        font-size: 12px;
    }
    .styled-table th {
        background-color: #4472C4;
        color: white;
        text-align: center;
    }
    .styled-table td {
        text-align: center;
        border: 1px solid #ccc;
        padding: 4px;
    }
    </style>
    </head>
    <body>
    <h1>Resumen de Cuenta - {{ cliente }}</h1>
    <p><strong>Fecha de reporte:</strong> {{ fecha }}</p>
    <p><strong>Total Facturado:</strong> ${{ total_importe|format_number }}</p>
    <p><strong>Total Pagado:</strong> ${{ total_pagado|format_number }}</p>
    <p><strong>Saldo Pendiente:</strong> ${{ total_saldo|format_number }}</p>
    <img src="{{ grafico }}" width="700">
    {{ tabla|safe }}
    </body></html>
    """)

    html = template_html.render(
        cliente=cliente,
        fecha=datetime.today().strftime("%Y-%m-%d"),
        total_importe=total_importe,
        total_pagado=total_pagado,
        total_saldo=total_saldo,
        grafico=grafico_path,
        tabla=tabla_html
    )

    # Guardar el PDF en la carpeta correcta
    pdf_path = f"{reportes_path}/{cliente}_reporte.pdf"
    with open(pdf_path, "wb") as f:
        pisa.CreatePDF(html, dest=f)

    # Confirmar que el PDF fue guardado
    print(f"PDF guardado en: {pdf_path}")

# Verificar si los archivos fueron creados
print("Archivos generados en:", reportes_path)

**AGOSTINA MARINI STASI**

Github: *https://github.com/agostinamarini*

LinkedIn: *www.linkedin.com/in/agostina-marini-1b18a31b3*